In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import copy
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()
rt.co_mgr.str_to_color_lu['r'] = '#ff0000'
rt.co_mgr.str_to_color_lu['b'] = '#0000ff'
rt.co_mgr.str_to_color_lu['g'] = '#08ff08'
rt.co_mgr.str_to_color_lu['o'] = '#e59866'

In [ ]:
df = pl.DataFrame({'fm':'a b c d e f g h h i i i'.split(),
                   'to':'b c d e f g h i i a a a'.split(),
                   'co':'r r b b g g g g b o o o'.split()})
pos = {'a':(0.0,0.0), 'b':(0.5,0.0), 'c':(1.0,0.0), 'd':(0.8,0.5),
       'e':(1.0,1.0), 'f':(0.5,1.0), 'g':(0.0,1.0), 'h':(0.4,0.7), 'i':(0.2,0.35)}
relates = [('fm','to')]
params  = {'df':df, 'relationships':relates, 'pos':pos, 'draw_labels':True, 'link_arrow':False, 
           'color_by':'co', 'link_color':'vary', 'link_size':8.0, 'bounds_percent':0.2, 'w':192, 'h':192}
params2 = copy.deepcopy(params)
params2['link_size'], params2['link_size_max'], params2['link_size_min'] = 'vary', 8.0, 4.0
params3 = copy.deepcopy(params)
params3['node_color'], params3['link_color'], params3['link_size'] = 'vary', '#000000', 1.0
rt.tile([rt.link(**params), rt.linkNode(**params), rt.link(**params2), rt.linkNode(**params2), rt.link(**params3), rt.linkNode(**params3)])

In [ ]:
#
# Basic Recipe for Handling Color within Polars
#
_df_ = df.with_columns(pl.col('co').map_elements(lambda x: rt.co_mgr.getColor(x), return_dtype=pl.String).alias('__color__'))
_df_ = _df_.with_columns(pl.lit(rt.co_mgr.getTVColor('data','default')).alias('__color_default__'))
_df_ = _df_.group_by(['fm','to']).agg(pl.col('__color__').n_unique().alias('__color_nuniq__'),
                                      pl.col('__color_default__').first(),
                                      pl.col('__color__').first().alias('__color_first__')) 
_df_ = _df_.with_columns((pl.when(pl.col('__color_nuniq__')==1).then(pl.col('__color_first__')).otherwise(pl.col('__color_default__'))).alias('__color_final__'))
# _df_

In [ ]:
rt.tile([rt.link(x_ins=32, y_ins=32, **params), rt.link(x_ins=32, y_ins=32, draw_performance=False, **params)])

In [ ]:
# rt.link(pl.DataFrame({'fm':['a','b','c'], 'to':[1, 2, 3]}), [('fm','to')]) # fails to work properly
_x_ = rt.linkNode(pl.DataFrame({'fm':[2, 3, 1], 'to':[1, 2, 3]}), [('fm','to')])
_x_

In [ ]:
_y_ = rt.linkNode(pl.DataFrame({'fm':['b','c','a'], 'to':['a','b','c']}), [('fm','to')])
_y_

In [ ]:
_x_.pos['a'], _y_.pos['a'] # note that these are the same (even though they should have been randomly generated on instance creation)

In [ ]:
#
# Attempting to replicate issue above (persistence of a default variable setting across class instance creations)
# ... appears that this only affects empty dictionaries...
#
class ABC:
    def __init__(self, req, opt={}):
        self.req = req
        self.opt = opt
        if 5 not in self.opt.keys(): self.opt[5] = random.random()

class ABCCreator:
    def __init__(self):
        pass
    def createAnABC(self, req, opt={}):
        _params_ = locals().copy()
        _params_.pop('self')
        return ABC(**_params_)

creator = ABCCreator()
abc = creator.createAnABC('abc')
xyz = creator.createAnABC('xyz')
abc.opt, xyz.opt

In [ ]:
x = ABC(3)
y = ABC(4)
x.opt, y.opt

In [ ]:
z = ABC(2)
z.opt